In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg, select_category_value_agg, cnt_encoding
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals, impute_avg
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-08-08 19:43:29,951 logger 19 [INFO]    [logger_func] start 
2018-08-08 19:43:30,019 logger 19 [INFO]    [logger_func] start 
2018-08-08 19:43:30,019 logger 19 [INFO]    [logger_func] start 


In [2]:
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
app = pd.read_csv('../data/application_train_test.csv')
birth = app['DAYS_BIRTH'].values
birth = birth - birth.min() +1
birth = np.log10(birth)
app['DAYS_BIRTH_log10'] = birth

pos = pd.read_csv('../data/POS_CASH_balance.csv')

pos['NAME_CONTRACT_STATUS'] = pos['NAME_CONTRACT_STATUS'].map(lambda x: 'Signed' if x=='Approved' or x=='XNA' else 'Returned to the store' if x=='Canceled' else x)

pos.sort_values(by=[unique_id, p_id], inplace=True)

# print(pos['NAME_CONTRACT_STATUS'].drop_duplicates())
# pos.head()

In [9]:
pos['CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@'] = (pos['CNT_INSTALMENT'] > pos['CNT_INSTALMENT_FUTURE']).astype(int)

In [3]:
prev = pd.read_csv('../data/previous_cleansing.csv')
prev = prev[[unique_id, p_id, 'AMT_APPLICATION', 'AMT_CREDIT', 'DAYS_DECISION']]
pp = pos.merge(prev, on=[unique_id, p_id], how='inner').merge(app[[unique_id, 'AMT_INCOME_TOTAL', 'DAYS_BIRTH_log10', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH']], on=unique_id, how='inner')
pp.sort_values(by=[unique_id, p_id, 'MONTHS_BALANCE'], ascending=False, inplace=True)
pp = row_number(df=pp, level=[unique_id, p_id])
# pos_pr.query("SK_ID_PREV==1743609").head(40)

In [20]:
pp = diff_feature(df=pp, first='MONTHS_BALANCE', second='DAYS_DECISION')
pp = diff_feature(df=pp, first='AMT_APPLICATION', second='AMT_CREDIT')
amt_list = ['AMT_CREDIT', 'AMT_CREDIT_diff_AMT_APPLICATION@']
for amt in amt_list:
    pp = division_feature(df=pp, first=amt, second='AMT_INCOME_TOTAL')
    pp = division_feature(df=pp, first=amt, second='DAYS_BIRTH_log10')
    pp = division_feature(df=pp, first=amt, second='CNT_INSTALMENT')

In [5]:
pp = diff_feature(df=pp, first='SK_DPD', second='SK_DPD_DEF')

In [6]:
pp.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,TARGET,AMT_APPLICATION,AMT_CREDIT,DAYS_DECISION,AMT_INCOME_TOTAL,DAYS_BIRTH_log10,DAYS_EMPLOYED,DAYS_ID_PUBLISH,row_no,SK_DPD_diff_SK_DPD_DEF@
0,2729207,456255,-13,3.0,0.0,Completed,0,0,0,58225.50,58545.00,-16.0,157500.0,2.447158,-42.0,-13,1,0
1,2729207,456255,-14,3.0,0.0,Active,0,0,0,58225.50,58545.00,-16.0,157500.0,2.447158,-42.0,-13,2,0
2,2729207,456255,-15,6.0,5.0,Active,0,0,0,58225.50,58545.00,-16.0,157500.0,2.447158,-42.0,-13,3,0
3,2729207,456255,-16,6.0,6.0,Active,0,0,0,58225.50,58545.00,-16.0,157500.0,2.447158,-42.0,-13,4,0
4,2631384,456255,-2,24.0,0.0,Completed,0,0,0,848598.93,910208.43,-26.0,157500.0,2.447158,-42.0,-13,1,0


In [39]:
pos.query("NAME_CONTRACT_STATUS=='Amortized debt'")[[unique_id, target]].drop_duplicates().groupby(target)[unique_id].count()

TARGET
-1    6
 0    6
 1    5
Name: SK_ID_CURR, dtype: int64

In [11]:
prefix = 'pos_'
# df = pp.query('MONTHS_BALANCE >= -3')
# df = pp.query('row_no <= 15')
df = pos

# num_list = get_numeric_features(data=pp, ignore=ignore_features)
# for x in ['AMT_APPLICATION', 'AMT_CREDIT', 'AMT_INCOME_TOTAL',  'DAYS_BIRTH_log10', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'row_no', 'MONTHS_BALANCE_diff_DAYS_DECISION@', 'DAYS_DECISION']:
#     num_list.remove(x)
# num_list = num_list + ['SK_DPD_diff_SK_DPD_DEF@']
num_list = ['CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@']

method_list = ['skew', 'sum', 'mean', 'std', 'max', 'min']
# num_list.remove('diff')
# num_list.remove('weight')
# num_list.remove('tmp')

In [12]:
' 1段階の集計 '
level = unique_id
num_list = num_list

select_level_agg(base=base, data=df, ignore_list=ignore_features,  level=level, method_list=method_list, num_list=num_list, prefix=prefix)

2018-08-08 19:51:07,665 logger 222 [INFO]    [select_level_agg] 
num: CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@
method: skew
['SK_ID_CURR'] 
2018-08-08 19:51:07,665 logger 222 [INFO]    [select_level_agg] 
num: CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@
method: skew
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-08-08 19:53:40,624 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@_skew@['SK_ID_CURR']'], dtype='object') 
2018-08-08 19:53:40,624 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@_skew@['SK_ID_CURR']'], dtype='object') 
2018-08-08 19:53:40,809 logger 230 [INFO]    [selec

2018-08-08 19:53:42,162 logger 222 [INFO]    [select_level_agg] 
num: CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@
method: mean
['SK_ID_CURR'] 
2018-08-08 19:53:42,162 logger 222 [INFO]    [select_level_agg] 
num: CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@
method: mean
['SK_ID_CURR'] 
2018-08-08 19:53:43,443 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@_mean@['SK_ID_CURR']'], dtype='object') 
2018-08-08 19:53:43,443 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@_mean@['SK_ID_CURR']'], dtype='object') 
2018-08-08 19:53:43,595 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-08 19:53:43,595 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-08 19:53:43,613 logger 30 [INFO]    [make_npy] 0.857143    21388
0.909091    168

2018-08-08 19:53:45,153 logger 222 [INFO]    [select_level_agg] 
num: CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@
method: max
['SK_ID_CURR'] 
2018-08-08 19:53:45,153 logger 222 [INFO]    [select_level_agg] 
num: CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@
method: max
['SK_ID_CURR'] 
2018-08-08 19:53:46,312 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@_max@['SK_ID_CURR']'], dtype='object') 
2018-08-08 19:53:46,312 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE@_max@['SK_ID_CURR']'], dtype='object') 
2018-08-08 19:53:46,517 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-08 19:53:46,517 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-08 19:53:46,555 logger 30 [INFO]    [make_npy] 1.0    337038
0.0       214
Name: CNT

In [6]:
' 1段階の集計 value別'
level = unique_id
num_list = num_list

select_category_value_agg(base=base, data=df, ignore_list=ignore_features,  level=level, cat_list=['NAME_CONTRACT_STATUS'], method_list=method_list, num_list=num_list, prefix=prefix)

NameError: name 'num_list' is not defined

In [5]:
' 1段階の重み付き平均 '
level = unique_id
label = 'MONTHS_BALANCE'
weight_list = [0.99, 0.97, 0.95]
feature_list = num_list

for num in feature_list:
    if num==label:continue
    for weight in weight_list:
        wavg = exp_weight_avg(data=df, level=level, weight=weight, label=label, value=num)
        result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)

In [3]:
' 2段階の集計 '
level = [unique_id, p_id]
num_list = num_list

for method in method_list:
    method_list_2 = method_list.copy()
    if method in ['sum', 'max', 'min']:
        method_list_2.remove(method)
    for num in num_list:
        tmp = df.groupby(level)[num].agg({num:method}).reset_index().rename(columns={num:f'{num}_{method}@'})
    
        num_list_2 = [col for col in tmp.columns if col.count('@')]
        select_level_agg(base=base, data=tmp, ignore_list=ignore_features,  level=unique_id, method_list=method_list_2, num_list=num_list_2, prefix=prefix)

In [4]:
' 重み付き平均と集計 '
level = [unique_id, p_id]
label = 'MONTHS_BALANCE'
weight_list = [0.99, 0.97, 0.95]
feature_list = num_list

for num in feature_list:
    if num==label:continue
    for weight in weight_list:
        wavg = exp_weight_avg(data=df, level=level, weight=weight, label=label, value=num)
        
        wavg_list = [wavg.name]
        wavg = wavg.to_frame().reset_index()
        select_level_agg(base=base, data=wavg, ignore_list=ignore_features, level=unique_id, method_list=method_list, num_list=wavg_list, prefix=prefix)

In [34]:
' カテゴリ変数をダミー化して平均（出現率）を取得する '
prefix = 'pos_'
dummie_col = [col for col in pos.columns if col.count('CONTRACT_STATUS_')]

dummie = get_dummies(df, cat_list)

dummie_avg = pos[[unique_id] + dummie_col].groupby(unique_id).mean().reset_index()
for col in dummie_avg.columns:
    if col in ignore_features:
        continue
    dummie_avg.rename(columns={col:f'{col}@'}, inplace=True)
dummie_avg.head()

result = base.merge(dummie_avg, on=unique_id, how='left').fillna(0)
make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)

NameError: name 'cat_list' is not defined